In [ ]:
!pip install -U bitsandbytes

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Loading checkpoints and tokenizer
checkpoint = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint)

# Lets infrerence this model

my_text = "what is the capital of india?"
input_ids = tokenizer(my_text, return_tensors='pt').input_ids
print(f"Here is the generated text : {tokenizer.decode(model.generate(input_ids, num_beams= 2, length_penalty=0.8, max_length=100)[0], skip_special_tokens=True)}")


# Lets check the memory of the model

print(f"The size of the model is : {round(model.get_memory_footprint()/1024**3, 2)}")
print(f"The precision for this model is : {next(model.parameters()).dtype}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pyth

Here is the generated text : Mumbai
The size of the model is : 0.29
The precision for this model is : torch.float32


In [3]:
# lets start Quantization

# Note: GPU is required for this process

from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_quant_type = 'nf4'
)

q_model = T5ForConditionalGeneration.from_pretrained(checkpoint, quantization_config=bnb_config)

# Lets check the memory of the model

print(f"The size of the model is : {round(q_model.get_memory_footprint()/1024**3, 2)}")
print(f"The precision for this model is : {next(q_model.parameters()).dtype}")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


The size of the model is : 0.13
The precision for this model is : torch.float16


In [4]:
# Lets infrerence this model

my_text = "what is the capital of india?"
input_ids = tokenizer(my_text, return_tensors='pt').input_ids
print(f"Here is the generated text : {tokenizer.decode(q_model.generate(input_ids, num_beams= 2, length_penalty=0.8, max_length=100)[0], skip_special_tokens=True)}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Here is the generated text : Mumbai
